# A/B Test Challenge



---

#### What is an A/B Test? 

It is a decision making support & research methodology that allow you to measure an impact of a change in a product (e.g.: a digital product). For this challenge you will analyse the data resulting of an A/B test performed on a digital product where a new set of sponsored ads are included.


#### Measure of success

Metrics are needed to measure the success of your product. They are typically split in the following categories: 

- __Enganged based metrics:__ number of users, number of downloads, number of active users, user retention, etc.

- __Revenue and monetization metrics:__ ads and affiliate links, subscription-based, in-app purchases, etc.

- __Technical metrics:__ service level indicators (uptime of the app, downtime of the app, latency).



---

## Metrics understanding

In this part you must analyse the metrics involved in the test. We will focus in the following metrics:

- Activity level + Daily active users (DAU).

- Click-through rate (CTR)

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import os

from statsmodels.stats.weightstats import ztest
from statsmodels.stats.weightstats import ttest_ind
from scipy import stats

import seaborn as sns
import matplotlib.pylab as plt

### Activity level

In the following part you must perform every calculation you consider necessary in order to answer the following questions:

- How many activity levels you can find in the dataset (Activity level of zero means no activity).

- What is the amount of users for each activity level.

- How many activity levels do you have per day and how many records per each activity level.

At the end of this section you must provide your conclusions about the _activity level_ of the users.

__Dataset:__ `activity_pretest.csv`

In [ ]:
# How to read several DF from several .csv included in a .zip file
zip_file = ZipFile('./diamonds_train_data.zip')
dfs = {text_file.filename: pd.read_csv(zip_file.open(text_file.filename))
       for text_file in zip_file.infolist()
       if text_file.filename.endswith('.csv')}

In [2]:
path = 'data/raw_data/'
sep = ","
file_type = 'csv'

content = os.listdir(path)

# With this function I can extract all of datasets in diferent dataframes. In order to understand each dataframe, I will 
# print the name, columns name and shape of each dataframe.
for name_file in content:
    if name_file.split('.')[1] == file_type:
        # Split the name file, extract the first part of the string and add '_df'. This will be the dataframe name.
        name_df = name_file.split('.')[0] + '_df'
        # Read csv as dataframe and store the data in a variable called as 'name_df' value
        exec(f'{name_df} = pd.read_csv(f"{path + name_file}", sep=",")')
        # Extract the column names of dataframe
        exec(f'columns_name = {name_df}.columns.values')
        # Extract the shape of dataframe
        exec(f'shape = {name_df}.shape')
        # Print the dataframe name, column names and dataframe shape
        print(name_df,'| column names: ', columns_name, '| df shape: ', shape)

activity_all_df | column names:  ['userid' 'dt' 'groupid' 'activity_level'] | df shape:  (3660000, 4)
activity_pretest_df | column names:  ['userid' 'dt' 'activity_level'] | df shape:  (1860000, 3)
ctr_all_df | column names:  ['userid' 'dt' 'groupid' 'ctr'] | df shape:  (2303408, 4)
ctr_pretest_df | column names:  ['userid' 'dt' 'ctr'] | df shape:  (950875, 3)


In [19]:
activity_pretest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1860000 entries, 0 to 1859999
Data columns (total 3 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   userid          object
 1   dt              object
 2   activity_level  int64 
dtypes: int64(1), object(2)
memory usage: 42.6+ MB


In [60]:
activity_pretest_df.head()

,userid,dt,activity_level
0,a5b70ae7-f07c-4773-9df4-ce112bc9dc48,2021-10-01,0
1,d2646662-269f-49de-aab1-8776afced9a3,2021-10-01,0
2,c4d1cfa8-283d-49ad-a894-90aedc39c798,2021-10-01,0
3,6889f87f-5356-4904-a35a-6ea5020011db,2021-10-01,0
4,dbee604c-474a-4c9d-b013-508e5a0e3059,2021-10-01,0


In [66]:
# How many activity levels you can find in the dataset (Activity level of zero means no activity).
level_activity = len(activity_pretest_df['activity_level'].unique())
level_activity

21

In [78]:
# What is the amount of users for each activity level.
activity_without_zero_df = activity_pretest_df[activity_pretest_df['activity_level']>0].reset_index()
activity_level_serie = activity_without_zero.groupby(['activity_level'])['userid'].count()
activity_level_serie.head()

activity_level
1    48732
2    49074
3    48659
4    48556
5    49227
Name: userid, dtype: int64

In [61]:
# How many activity levels do you have per day and how many records per each activity level.
activity_levels_day_df = activity_without_zero.groupby(['dt', 'activity_level']).size().reset_index(name='count')
activity_levels_day_df.head()

,dt,activity_level,count
0,2021-10-01,1,1602
1,2021-10-01,2,1507
2,2021-10-01,3,1587
3,2021-10-01,4,1551
4,2021-10-01,5,1586


### Daily active users (DAU)

![ab_test](./img/user_activity_ab_testing.JPG)


The daily active users (DAU) refers to the amount of users that are active per day (activity level of zero means no activity). You must perform the calculation of this metric and provide your insights about it.

__Dataset:__ `activity_pretest.csv`

In [83]:
# Transrform 'dt' column to datetime
activity_without_zero_df['dt'] = pd.to_datetime(activity_without_zero['dt'])
activity_without_zero_df

,index,userid,dt,activity_level
0,909125,428070b0-083e-4c0e-8444-47bf91e99fff,2021-10-01,1
1,909126,93370f9c-56ef-437f-99ff-cb7c092d08a7,2021-10-01,1
2,909127,0fb7120a-53cf-4a51-8b52-bf07b8659bd6,2021-10-01,1
3,909128,ce64a9d8-07d9-4dca-908d-5e1e4568003d,2021-10-01,1
4,909129,e08332f0-3a5c-4ed2-b957-87e464e89b97,2021-10-01,1
...,...,...,...,...
950870,1859995,200d65e6-b1ce-4a47-8c2b-946db5c5a3a0,2021-10-31,20
950871,1859996,535dafe4-de7c-4b56-acf6-aa94f21653bc,2021-10-31,20
950872,1859997,0428ca3c-e666-4ef4-8588-3a2af904a123,2021-10-31,20
950873,1859998,a8cd1579-44d4-48b3-b3d6-47ae5197dbc6,2021-10-31,20


In [86]:
# Calcular Usuarios Activos Diarios (DAU)
dau_serie = activity_without_zero.groupby('dt')['userid'].nunique()
dau_serie.head()

dt
2021-10-01    30634
2021-10-02    30775
2021-10-03    30785
2021-10-04    30599
2021-10-05    30588
Name: userid, dtype: int64

### Click-through rate (CTR)

![ab_test](./img/ad_click_through_rate_ab_testing.JPG)

Click-through rate (CTR) refers to the percentage of clicks that the user perform from the total amount ads showed to that user during a certain day. You must perform the analysis of this metric (e.g.: average CTR per day) and provide your insights about it.

__Dataset:__ `ctr_pretest.csv`

In [88]:
ctr_pretest_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 950875 entries, 0 to 950874
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   userid  950875 non-null  object 
 1   dt      950875 non-null  object 
 2   ctr     950875 non-null  float64
dtypes: float64(1), object(2)
memory usage: 21.8+ MB


In [87]:
ctr_pretest_df.head()

,userid,dt,ctr
0,4b328144-df4b-47b1-a804-09834942dce0,2021-10-01,34.28
1,34ace777-5e9d-40b3-a859-4145d0c35c8d,2021-10-01,34.67
2,8028cccf-19c3-4c0e-b5b2-e707e15d2d83,2021-10-01,34.77
3,652b3c9c-5e29-4bf0-9373-924687b1567e,2021-10-01,35.42
4,45b57434-4666-4b57-9798-35489dc1092a,2021-10-01,35.04


In [92]:
# your-code
ctr_pretest_df['dt'] = pd.to_datetime(ctr_pretest_df['dt'])

# Calcular el CTR medio por día
ctr_mean_da_serie = ctr_pretest_df.groupby('dt')['ctr'].mean()

# Imprimir el resultado
print('CTR mean/day:')
print(ctr_mean_day)

# Estadísticas descriptivas del CTR
ctr_statistics = ctr_pretest_df['ctr'].describe()

# Imprimir las estadísticas descriptivas
print('\nEstadísticas Descriptivas del CTR:')
print(ctr_statistics)

CTR mean/day:
dt
2021-10-01    32.993446
2021-10-02    32.991664
2021-10-03    32.995086
2021-10-04    32.992995
2021-10-05    33.004375
2021-10-06    33.018564
2021-10-07    32.988500
2021-10-08    32.998654
2021-10-09    33.005082
2021-10-10    33.007134
2021-10-11    32.990300
2021-10-12    32.996166
2021-10-13    32.984248
2021-10-14    32.999878
2021-10-15    33.008517
2021-10-16    32.991025
2021-10-17    33.001919
2021-10-18    33.007763
2021-10-19    33.001511
2021-10-20    33.004632
2021-10-21    32.997566
2021-10-22    33.006785
2021-10-23    33.012228
2021-10-24    32.984093
2021-10-25    32.990223
2021-10-26    33.014248
2021-10-27    33.007045
2021-10-28    33.005711
2021-10-29    33.004230
2021-10-30    33.016430
2021-10-31    32.987515
Name: ctr, dtype: float64

Estadísticas Descriptivas del CTR:
count    950875.000000
mean         33.000242
std           1.731677
min          30.000000
25%          31.500000
50%          33.000000
75%          34.500000
max          36.

---

## Pretest metrics 

In this section you will perform the analysis of the metrics using the dataset that includes the result for the test and control groups, but only for the pretest data (i.e.: prior to November 1st, 2021). You must provide insights about the metrics (__Activity level__, __DAU__ and __CTR__) and also perform an hyphotesis test in order to determine whether there is any statistical significant difference between the groups prior to the start of the experiment. You must try different approaches (i.e.: __z-test__ and __t-test__) and compare the results.


__Datasets:__ `activity_all.csv`, `ctr_all.csv`

In [3]:
# Merge the two dataframes
data_df = pd.merge(activity_all_df, ctr_all_df, on=['userid', 'dt'], how='inner')
# Convert 'dt' column to datetime if not already done
data_df['dt'] = pd.to_datetime(data_df['dt'])
data_df

,userid,dt,groupid_x,activity_level,groupid_y,ctr
0,428070b0-083e-4c0e-8444-47bf91e99fff,2021-10-01,1,1,1,33.55
1,93370f9c-56ef-437f-99ff-cb7c092d08a7,2021-10-01,1,1,1,35.89
2,0fb7120a-53cf-4a51-8b52-bf07b8659bd6,2021-10-01,1,1,1,32.95
3,ce64a9d8-07d9-4dca-908d-5e1e4568003d,2021-10-01,0,1,0,31.12
4,e08332f0-3a5c-4ed2-b957-87e464e89b97,2021-10-01,1,1,1,34.21
...,...,...,...,...,...,...
2303403,f0126b50-ad74-4480-9250-41b50a408932,2021-11-30,0,20,0,32.96
2303404,6ffe1efe-2e5d-427f-95ff-cc862c46c798,2021-11-30,1,20,1,40.52
2303405,f2073207-25dd-4127-a893-b70106d5ead7,2021-11-30,0,20,0,30.88
2303406,0416f2be-3ab8-481b-873c-3678b4705ecf,2021-11-30,1,20,1,40.80


In [4]:
# Pretest data extraction
pretest_df = data_df[data_df['dt'] < '2021-11-01']

# Separate data for test and control groups
pre_test_group = pretest_df[pretest_df['groupid_x'] == 1]   # groupid==1 -> test group
pre_control_group = pretest_df[pretest_df['groupid_x'] == 0]   # groupid==0 -> control group

# Analysis of Activity Level
pre_activity_test = pre_test_group['activity_level']
pre_activity_control = pre_control_group['activity_level']

# Analysis of DAU (Daily Active Users)
pre_dau_test = pre_test_group.groupby('dt')['userid'].nunique()
pre_dau_control = pre_control_group.groupby('dt')['userid'].nunique()

# Analysis of CTR (Click-Through Rate)
pre_ctr_test = pre_test_group.groupby('dt')['ctr'].mean()
pre_ctr_control = pre_control_group.groupby('dt')['ctr'].mean()

# HYPOTHESIS
# For each of the tests to be performed, the hypotheses for each case will be:
# H0: u_test = u_control
# H1: u_test != u_control
# This assumption is the same for activity level, dau and ctr data.

# T-TEST
t_stat_activity, p_value_activity = stats.ttest_ind(pre_activity_test, pre_activity_control, equal_var=False)
t_stat_dau, p_value_dau = stats.ttest_ind(pre_dau_test, pre_dau_control, equal_var=False)
t_stat_ctr, p_value_ctr = stats.ttest_ind(pre_ctr_test, pre_ctr_control, equal_var=False)
# Print results
print('T-test:')
print(f'Activity Level: {t_stat_activity}, p-value: {p_value_activity}')
print(f'DAU: {t_stat_dau}, p-value: {p_value_dau}')
print(f'CTR: {t_stat_ctr}, p-value: {p_value_ctr} \n')

# Z-TEST
# Alternatively, perform Z-tests
z_score_activity, p_value_activity_z = ztest(pre_activity_test, pre_activity_control)
z_score_dau, p_value_dau_z = ztest(pre_dau_test, pre_dau_control)
z_score_ctr, p_value_ctr_z = ztest(pre_ctr_test, pre_ctr_control)
# Print results for Z-tests
print('Z-test:')
print(f'Activity Level: {z_score_activity}, p-value: {p_value_activity_z}')
print(f'DAU: {z_score_dau}, p-value: {p_value_dau_z}')
print(f'CTR: {z_score_ctr}, p-value: {p_value_ctr_z}')

T-test:
Activity Level: 0.2752136824733843, p-value: 0.7831521354232784
DAU: 1.4121065242323185, p-value: 0.16309165273757295
CTR: -0.38136356232323276, p-value: 0.70428193587848 

Z-test:
Activity Level: 0.27521370941856227, p-value: 0.7831520549245693
DAU: 1.4121065242323187, p-value: 0.15791859802311015
CTR: -0.3813635623232328, p-value: 0.7029334947610391


In this analysis, there is no statistically significant difference in Activity Level, DAU, and CTR between the test and control groups. The p-values for all metrics are greater than the significance level of 0.05, indicating that you do not have enough evidence to reject the null hypothesis.

---

## Experiment metrics 

In this section you must perform the same analysis as in the previous section, but using the data generated during the experiment (i.e.: after November 1st, 2021). You must provide insights about the metrics (__Activity level__, __DAU__ and __CTR__) and also perform an hyphotesis test in order to determine whether there is any statistical significant difference between the groups during the experiment. You must try different approaches (i.e.: __z-test__ and __t-test__) and compare the results.


__Datasets:__ `activity_all.csv`, `ctr_all.csv`

In [22]:
# your-code
# Experiment data extraction
experiment_df = data_df[data_df['dt'] > '2021-11-01']

# Separate data for test and control groups
exp_test_group = experiment_df[experiment_df['groupid_x'] == 1]   # groupid==1 -> test group
exp_control_group = experiment_df[experiment_df['groupid_x'] == 0]   # groupid==0 -> control group

# Analysis of Activity Level
exp_activity_test = exp_test_group['activity_level']
exp_activity_control = exp_control_group['activity_level']

# Analysis of DAU (Daily Active Users)
exp_dau_test = exp_test_group.groupby('dt')['userid'].nunique()
exp_dau_control = exp_control_group.groupby('dt')['userid'].nunique()

# Analysis of CTR (Click-Through Rate)
exp_ctr_test = exp_test_group.groupby('dt')['ctr'].mean()
exp_ctr_control = exp_control_group.groupby('dt')['ctr'].mean()

# HYPOTHESIS
# For each of the tests to be performed, the hypotheses for each case will be:
# H0: u_test = u_control
# H1: u_test != u_control
# This assumption is the same for activity level, dau and ctr data.

# T-TEST
t_stat_activity, p_value_activity = stats.ttest_ind(exp_activity_test, exp_activity_control, equal_var=False)
t_stat_dau, p_value_dau = stats.ttest_ind(exp_dau_test, exp_dau_control, equal_var=False)
t_stat_ctr, p_value_ctr = stats.ttest_ind(exp_ctr_test, exp_ctr_control, equal_var=False)
# Print results
print('T-test:')
print(f'Activity Level: {t_stat_activity}, p-value: {p_value_activity}')
print(f'DAU: {t_stat_dau}, p-value: {p_value_dau}')
print(f'CTR: {t_stat_ctr}, p-value: {p_value_ctr} \n')

# Z-TEST
# Alternatively, perform Z-tests
z_score_activity, p_value_activity_z = ztest(exp_activity_test, exp_activity_control)
z_score_dau, p_value_dau_z = ztest(exp_dau_test, exp_dau_control)
z_score_ctr, p_value_ctr_z = ztest(exp_ctr_test, exp_ctr_control)
# Print results for Z-tests
print('Z-test:')
print(f'Activity Level: {z_score}, p-value: {p_value_activity_z}')
print(f'DAU: {z_score_dau}, p-value: {p_value_dau_z}')
print(f'CTR: {z_score_ctr}, p-value: {p_value_ctr_z}')

T-test:
Activity Level: -0.3637933588130946, p-value: 0.716012404454498
DAU: 193.3233880088953, p-value: 8.030296925465817e-46
CTR: 1801.608983707, p-value: 4.579459767338931e-123 

Z-test:
Activity Level: -0.3639225719576132, p-value: 0.7159158287460562
DAU: 193.32338800889534, p-value: 0.0
CTR: -1801.608983707, p-value: 0.0


In this case, at the activity level, p_value is greater than 0.05, so the null hypothesis cannot be rejected.

But for the DAU and CTR values, values much smaller than 0.05 are obtained from both tests, so the null hypothesis can be rejected and the alternative hypothesis is accepted with 95% confidence.

---

## Conclusions

Please provide your conclusions after the analyses and your recommendation whether we may or may not implement the changes in the digital product.

The different series for the control and test groups for the pretest and experimental populations have already been obtained above. This has been obtained for the activity level, DAU and CTR.

A t-test and z-test analysis of the 4 groups will be performed:
- Analysis 1:
  - control group before the experiment (pre_< >_control).
  - experimental group before the experiment (pre_< >_test).

- Analysis 2: 
  - control group before experiment (pre_< >_control).
  - control group during the experiment (exp_< >_control).

- Analysis 3:
  - experimental group before experiment (pre_< >_test).
  - experimental group during the experiment (exp_< >_test).

- Analysis 4:
  - control group during the experiment (exp_< >_control).
  - experimental group during the experiment (exp_< >_test).
  
This will be carried out for each of the 3 calculated parameters (activity level, DAU and CTR.

##### HYPOTHESIS
For each of the tests to be performed, the hypotheses for each case will be:
- H0: u_test = u_control
- H1: u_test != u_control

This assumption is the same for activity level, dau and ctr data.

In [29]:
# Analysis of Activity Level
# These are the series we are going to use: 'pre_activity_control', 'pre_activity_test', 'exp_activity_control' and 
# 'exp_activity_test'

# Analysis 1
#t_stat_1, p_value_1 = stats.ttest_ind(pre_activity_control, pre_activity_test, equal_var=False)
#z_score_1, p_value_z_1 = ztest(pre_activity_control, pre_activity_test)
t_stat_1, p_value_1 = stats.ttest_ind(pre_activity_test, pre_activity_control, equal_var=False)
z_score_1, p_value_z_1 = ztest(pre_activity_test, pre_activity_control)

# Analysis 2
#t_stat_2, p_value_2 = stats.ttest_ind(pre_activity_control, exp_activity_control, equal_var=False)
#z_score_2, p_value_z_2 = ztest(pre_activity_control, exp_activity_control)
t_stat_2, p_value_2 = stats.ttest_ind(exp_activity_control, pre_activity_control, equal_var=False)
z_score_2, p_value_z_2 = ztest(pre_activity_control, exp_activity_control)

# Analysis 3
t_stat_3, p_value_3 = stats.ttest_ind(exp_activity_test, pre_activity_test, equal_var=False)
z_score_3, p_value_z_3 = ztest(exp_activity_test, pre_activity_test)

# Analysis 4
#t_stat_4, p_value_4 = stats.ttest_ind(exp_activity_test, exp_activity_control, equal_var=False)
#z_score_4, p_value_z_4 = ztest(exp_activity_test, exp_activity_control)
t_stat_4, p_value_4 = stats.ttest_ind(exp_activity_test, exp_activity_control, equal_var=False)
z_score_4, p_value_z_4 = ztest(exp_activity_test, exp_activity_control)

# Print results
print('Analysis 1:')
print(f'T-test: {t_stat_1}, p-value: {p_value_1}')
print(f'Z-test: {z_score_1}, p-value: {p_value_z_1}')

print('\nAnalysis 2:')
print(f'T-test: {t_stat_2}, p-value: {p_value_2}')
print(f'Z-test: {z_score_2}, p-value: {p_value_z_2}')

print('\nAnalysis 3:')
print(f'T-test: {t_stat_3}, p-value: {p_value_3}')
print(f'Z-test: {z_score_3}, p-value: {p_value_z_3}')

print('\nAnalysis 4:')
print(f'T-test: {t_stat_4}, p-value: {p_value_4}')
print(f'Z-test: {z_score_4}, p-value: {p_value_z_4}')

Analysis 1:
T-test: 0.2752136824733843, p-value: 0.7831521354232784
Z-test: 0.27521370941856227, p-value: 0.7831520549245693

Analysis 2:
T-test: -0.16162265932661785, p-value: 0.8716030474232682
Z-test: 0.16162516700377666, p-value: 0.8716010374796206

Analysis 3:
T-test: -0.8655449305747378, p-value: 0.38673997708842356
Z-test: -0.865608554339661, p-value: 0.38670486288869665

Analysis 4:
T-test: -0.3637933588130946, p-value: 0.716012404454498
Z-test: -0.3639225719576132, p-value: 0.7159158287460562


For the activity level analysis, we cannot reject the null hypothesis in any case, since all combinations (for t-test and z-test) the p-value is greater than alpha (0.05).

In [28]:
# Analysis of DAU (Daily Active Users)
# These are the series we are going to use: 'pre_dau_control', 'pre_dau_test', 'exp_dau_control' and 'exp_dau_test'

# Analysis 1
#t_stat_1_dau, p_value_1_dau = stats.ttest_ind(pre_dau_control, pre_dau_test, equal_var=False)
#z_score_1_dau, p_value_z_1_dau = ztest(pre_dau_control, pre_dau_test)
t_stat_1_dau, p_value_1_dau = stats.ttest_ind(pre_dau_test, pre_dau_control, equal_var=False)
z_score_1_dau, p_value_z_1_dau = ztest(pre_dau_test, pre_dau_control)

# Analysis 2
#t_stat_2_dau, p_value_2_dau = stats.ttest_ind(pre_dau_control, exp_dau_control, equal_var=False)
#z_score_2_dau, p_value_z_2_dau = ztest(pre_dau_control, exp_dau_control)
t_stat_2_dau, p_value_2_dau = stats.ttest_ind(exp_dau_control, pre_dau_control, equal_var=False)
z_score_2_dau, p_value_z_2_dau = ztest(exp_dau_control, pre_dau_control)

# Analysis 3
#t_stat_3_dau, p_value_3_dau = stats.ttest_ind(pre_dau_test, exp_dau_test, equal_var=False)
#z_score_3_dau, p_value_z_3_dau = ztest(pre_dau_test, exp_dau_test)
t_stat_3_dau, p_value_3_dau = stats.ttest_ind(exp_dau_test, pre_dau_test, equal_var=False)
z_score_3_dau, p_value_z_3_dau = ztest(exp_dau_test, pre_dau_test)

# Analysis 4
#t_stat_4_dau, p_value_4_dau = stats.ttest_ind(exp_dau_control, exp_dau_test, equal_var=False)
#z_score_4_dau, p_value_z_4_dau = ztest(exp_dau_control, exp_dau_test)
t_stat_4_dau, p_value_4_dau = stats.ttest_ind(exp_dau_test, exp_dau_control, equal_var=False)
z_score_4_dau, p_value_z_4_dau = ztest(exp_dau_test, exp_dau_control)
# Print results
print('Analysis 1:')
print(f'T-test: {t_stat_1_dau}, p-value: {p_value_1_dau}')
print(f'Z-test: {z_score_1_dau}, p-value: {p_value_z_1_dau}')

print('\nAnalysis 2:')
print(f'T-test: {t_stat_2_dau}, p-value: {p_value_2_dau}')
print(f'Z-test: {z_score_2_dau}, p-value: {p_value_z_2_dau}')

print('\nAnalysis 3:')
print(f'T-test: {t_stat_3_dau}, p-value: {p_value_3_dau}')
print(f'Z-test: {z_score_3_dau}, p-value: {p_value_z_3_dau}')

print('\nAnalysis 4:')
print(f'T-test: {t_stat_4_dau}, p-value: {p_value_4_dau}')
print(f'Z-test: {z_score_4_dau}, p-value: {p_value_z_4_dau}')

Analysis 1:
T-test: 1.4121065242323185, p-value: 0.16309165273757295
Z-test: 1.4121065242323187, p-value: 0.15791859802311015

Analysis 2:
T-test: 6.342337690893999, p-value: 4.651085116073182e-07
Z-test: 6.537177944284021, p-value: 6.269044592063694e-11

Analysis 3:
T-test: 842.248392345456, p-value: 1.2098537064782221e-82
Z-test: 820.4084012519231, p-value: 0.0

Analysis 4:
T-test: 193.3233880088953, p-value: 8.030296925465817e-46
Z-test: 193.32338800889534, p-value: 0.0


For the activity level analysis, the only case in which we cannot reject the null hypothesis is for the pretest and control values. For all other combinations, the null hypothesis is rejected with 95% confidence. 

For the pretest data sample, it can be stated that statistically there is no variation (at 95%) between the control sample and the study sample.
But for the data sample during the experiment this is not the case, rejecting the null hypothesis, there is statistical evidence of a difference. It can be said that this difference is much smaller (according to z-score) than between the cross samples (pretest and during the experiment).

In [30]:
# Analysis of CTR (Click-Through Rate)
# These are the series we are going to use: 'pre_ctr_control', 'pre_ctr_test', 'exp_ctr_control' and 'exp_ctr_test'

# Analysis 1
#t_stat_1_ctr, p_value_1_ctr = stats.ttest_ind(pre_ctr_control, pre_ctr_test, equal_var=False)
#z_score_1_ctr, p_value_z_1_ctr = ztest(pre_ctr_control, pre_ctr_test)
t_stat_1_ctr, p_value_1_ctr = stats.ttest_ind(pre_ctr_test, pre_ctr_control, equal_var=False)
z_score_1_ctr, p_value_z_1_ctr = ztest(pre_ctr_test, pre_ctr_control)

# Analysis 2
t_stat_2_ctr, p_value_2_ctr = stats.ttest_ind(exp_ctr_control, pre_ctr_control, equal_var=False)
z_score_2_ctr, p_value_z_2_ctr = ztest(exp_ctr_control, pre_ctr_control)

# Analysis 3
t_stat_3_ctr, p_value_3_ctr = stats.ttest_ind(exp_ctr_test, pre_ctr_test, equal_var=False)
z_score_3_ctr, p_value_z_3_ctr = ztest(exp_ctr_test, pre_ctr_test)

# Analysis 4
#t_stat_4_ctr, p_value_4_ctr = stats.ttest_ind(exp_ctr_control, exp_ctr_test, equal_var=False)
#z_score_4_ctr, p_value_z_4_ctr = ztest(exp_ctr_control, exp_ctr_test)
t_stat_4_ctr, p_value_4_ctr = stats.ttest_ind(exp_ctr_test, exp_ctr_control, equal_var=False)
z_score_4_ctr, p_value_z_4_ctr = ztest(exp_ctr_test, exp_ctr_control)

# Print results
print('Analysis 1:')
print(f'T-test: {t_stat_1_ctr}, p-value: {p_value_1_ctr}')
print(f'Z-test: {z_score_1_ctr}, p-value: {p_value_z_1_ctr}')

print('\nAnalysis 2:')
print(f'T-test: {t_stat_2_ctr}, p-value: {p_value_2_ctr}')
print(f'Z-test: {z_score_2_ctr}, p-value: {p_value_z_2_ctr}')

print('\nAnalysis 3:')
print(f'T-test: {t_stat_3_ctr}, p-value: {p_value_3_ctr}')
print(f'Z-test: {z_score_3_ctr}, p-value: {p_value_z_3_ctr}')

print('\nAnalysis 4:')
print(f'T-test: {t_stat_4_ctr}, p-value: {p_value_4_ctr}')
print(f'Z-test: {z_score_4_ctr}, p-value: {p_value_z_4_ctr}')

Analysis 1:
T-test: -0.38136356232323276, p-value: 0.70428193587848
Z-test: -0.3813635623232328, p-value: 0.7029334947610391

Analysis 2:
T-test: -1.0224515468417852, p-value: 0.31084696164636727
Z-test: -1.0170333682413653, p-value: 0.3091375559005337

Analysis 3:
T-test: 1688.5606129418622, p-value: 2.8125867707093073e-122
Z-test: 1663.6447491376293, p-value: 0.0

Analysis 4:
T-test: 1801.608983707, p-value: 4.579459767338931e-123
Z-test: 1801.608983707, p-value: 0.0


Finally, in the case of the analysis of the CTR data. The null hypothesis is accepted for the first two combinations and rejected with 95% confidence for the last two (where the test data population is used during the experiment).

#### CONCLUSION

It can be said that statistically the level of activity does not change and that for the same level of activity, a statistical increase in the CTR values is observed using these data.

Although it is true that from the statistical analysis of the daily activity per user, it is more difficult to draw conclusions, because there is a statistical difference between the pretest control samples and during the experiment.

In [ ]:
'''
HELP:
groupid==0 -> grupo de control
groupid==1 -> grupo del experimento

Hay 4 grupos:
    - grupo control antes del experimento
    - grupo de experimentación antes del experimento
    - grupo de control durante el experimento
    - grupo de experimentación durante el experimento

Caso 1 (el primer z_score entre): p_value=0.705 - Estos dos deberían ser iguales y lo son. No podemos rechazar H0
    H0=medias iguales | H1=son distintas
    - grupo control antes del experimento
    - grupo de experimentación antes del experimento
    
Caso 2 (el primer z_score entre): p_value=0.26 - No podemos rechazar H0
    - grupo control antes del experimento
    - grupo de control durante el experimento
    
Caso 3 (el primer z_score entre): p_value=0.0 - Rechazamos H0
    - grupo de experimentación antes del experimento
    - grupo de experimentación durante el experimento
    
Caso 4 (el primer z_score entre): p_value=0.0 - Rechazamos H0
    - grupo de control durante el experimento
    - grupo de experimentación durante el experimento
'''

---